<div style="margin: 0 auto 30px; height: 60px; border: 2px solid gray; border-radius: 6px;">
  <div style="float: left;"><img src="img/epfl.png" /></div>
  <div style="float: right; margin: 20px 30px 0; font-size: 10pt; font-weight: bold;"><a href="https://moodle.epfl.ch/course/view.php?id=18253">COM202 - Signal Processing</a></div>
</div>
<div style="clear: both; font-size: 30pt; font-weight: bold; color: #483D8B;">
    Lab 3: Time-frequency analysis of DTMF signals
</div>

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as sp
import IPython
from scipy.io import wavfile
import os
import csv

In [ ]:
plt.rcParams["figure.figsize"] = (13,2.5)

# Introduction

<div style="float: right; margin: 0 10px 0 30px;"><img src="img/phone.jpg" width="150"></div>

DTMF (Dual-Tone Multi-Frequency) is a signaling protocol used to transmit simple numeric information over the frequency band provided by analog telephone lines, that is, between 300 Hz and 3400 Hz. When you use the keypad of an analog phone such as the one shown on the right, the sequence of dialed digits is transmitted to the phone company's switches in the form of audible _dial tones_. Today, cell phones and office phones are directly connected to digital networks and therefore no longer use DTMF for dialing. But DTMF is still frequently used in automated attendant systems (i.e., those phone menus where you are told to "press 1 to talk to customer service" etc.)

Here you can listen to the signal corresponding to the sequence `123##45`                   :

In [ ]:
def read_test_file(filename: str) -> np.ndarray:
    # helper function to load a DTMF test file
    fs, x = wavfile.read(os.path.join('data', filename))    
    # normalize audio data to [-1, 1] if necessary
    if x.dtype is np.dtype(np.int16) :   
        x = x / 32767.0
    return fs, x

In [ ]:
Fs, dtmf_signal = read_test_file('dtmf.wav')
IPython.display.Audio(dtmf_signal, rate=Fs)

## Time-Frequency Analysis of a DTMF signal

A DTMF signal carries information both in time and in frequency: each dialed digit is encoded via two frequency values while the order in which the digits are dialed is encoded by the order in which the frequency pairs appear over time. But this creates a problem: if we look at the spectrum of the *entire* DTMF signal we can easily see which digits have been dialed by looking at the frequency peaks but we won't be able to determine their order and if a digit has been dialed multiple times.

For instance, these are the magnitude spectra of the DTMF signals corresponding to the sequences `159` and `915915`:

In [ ]:
# A helper function to compute the square magnitude of the DFT of a signal, together with a
#  set of frequency indexes in hertz

def spectral_power(x: np.ndarray, Fs: float) -> (np.ndarray, np.ndarray):
    L = len(x) // 2
    return np.linspace(0, Fs/2, L+1), np.abs(np.fft.fft(x)[:L+1]) ** 2 / len(x)

In [ ]:
for n, k in enumerate(['159', '915915']):
    Fs, x = read_test_file(f'keys_{k}.wav')
    plt.subplot(1,2,n+1)
    plt.plot(*spectral_power(x, Fs))
    plt.title(f'dialed digits: {k}');

On the other hand, if we look at the two signals in the time domain, we can clearly see the number and the order of the dialed digits but the frequencies of each DTMF tone are not easily identifiable:

In [ ]:
for n, k in enumerate(['159', '915915']):
    Fs, x = read_test_file(f'keys_{k}.wav')
    plt.subplot(1,2,n+1)
    plt.plot(np.arange(0, len(x)) / Fs, x)
    plt.title(f'dialed digits: {k}');

# The Short-Time Fourier Transform

If we are interested in how the *local* spectral properties of a signal evolve over time, we can split the signal into short consecutive chunks of length $N$ samples and compute the DFT of each chunk. The result is the two-dimensional sequence
$$
  S[c, k] = \sum_{n = 0}^{N-1} x[cN+n] e^{-j\frac{2\pi}{N}nk}.
$$
where $k \in [0, N-1]$ is the frequency index and $c \in \mathbb{Z}$ is chunk index; for a given value of $c$, the STFT returns the DFT coefficients for the finite-length signal $\begin{bmatrix} x[cN] & x[cN+1] & \ldots & x[cN + N - 1] \end{bmatrix}^T$.

## The spectrogram

In general, the STFT of a signal is represented graphically as an image in which the color of the pixel at coordinates $(c, k)$ is a function of $10\log_{10}(|S[c, k]|^2)$, the square magnitude of the STFT expressed in dB; typically, small values are associated to darker colors and large values to bright colors (or grayscale levels). In most practical applications the input signal is real-valued and therefore only the first $N/2$ DFT coefficients are displayed for each chunk. Since the sampling frequency $F_s$ is known, the image axes are labeled in seconds and in hertz. 

The STFT can be computed via the SciPy function `spectrogram`, and displayed with Matplotlib's matrix visualization function `colormesh`.

Here are the spectrograms of the previous DTMF signals, where now both the time and the frequency information are clearly identifiable.

In [ ]:
for n, k in enumerate(['159', '915915']):
    Fs, x = read_test_file(f'keys_{k}.wav')
    plt.subplot(1,2,n+1)
    f, t, S = sp.spectrogram(x, Fs)
    plt.pcolormesh(t, f, 10*np.log10(S + 1e-16));
    plt.title(f'dialed digits: {k}');

## Time-frequency resolution

When computing a STFT the main design parameter is the analysis window and, in particular, its length. A long window will produce a so-called *narrowband* spectrogram whereas a short window will produce a *wideband* spectrogram. Recall that the frequency resolution of an $N$-point DFT is $\Delta_\omega = 2\pi/N$ radians or, equivalently, $\Delta_f = F_s/N$ Hz if the sampling frequency is known. Therefore

if the analysis window is long:
 * more DFT points per chunk $\Rightarrow$ finer frequency resolution
 * more samples under the window $\Rightarrow$ more things can happen under the window $\Rightarrow$ lower time resolution

if the analysis window is short:
 * more chunks in total $\Rightarrow$ mroe precise time localization of spectral changes
 * fewer DFT points per chunk $\Rightarrow$ lower frequency resolution

In the following comparison you can see how in a narrowband spectrogram the frequency lines associated to the tones are much more precise than in a wideband spectrogram but, as a tradeoff, in a wideband spectrogram the start and stop instant of each tone burst are localized much more precisely.

In [ ]:
for n, win_len_ms, st in [(1, 80, 'narrowband'), (2, 10, 'wideband')]:
    f, t, S = sp.spectrogram(dtmf_signal, Fs, nperseg=int(win_len_ms / 1000 * Fs))
    plt.subplot(1,2,n)
    plt.pcolormesh(t, f, 10*np.log10(S + 1e-16));
    plt.title(f'{win_len_ms}ms window, {st} spectrogram');

# Implementing DTMF with DSP

DTMF was developed in the late 1950s and the first commercial DTMF phones hit the market in the 1960s. At the time, the system was implemented using analog hardware and the various frequencies were generated by a set of individual electronic oscillators.

<div style="float: left; margin: 0px;"><img src="img/mt8870.jpg" width="150"></div>

Obviously this is no longer the case and today DTMF signals are generated and decoded by dedicated (and extremely inexpensive) [DSP chips](https://pdf.datasheetcatalog.com/datasheets/228/268107_DS.pdf). In this notebook we will implement our own digital DTMF algorithms but, before anything else, let's review the relationship between the DTMF frequency values in Hz as specified by the standard and the digital frequencies that we will need to use in discrete time. This is relatively straightforward even without any formal knowledge of sampling and interpolation since all the signals involved are pure sinusoids.

## DTMF specifications

In DTMF the phone's keypad is arranged in a $4\times 3$ grid and each key is associated to a unique *pair* frequencies, as shown by this table:


|            | **1209 Hz** | **1336 Hz** | **1477 Hz** |
|------------|:-----------:|:-----------:|:-----------:|
| **697 Hz** |      1      |      2      |      3      |
| **770 Hz** |      4      |      5      |      6      |
| **852 Hz** |      7      |      8      |      9      |
| **941 Hz** |      *      |      0      |      #      |


When a key is pressed, two oscillators operating at the frequencies associated to the key send their output over the phone line. For instance, if the digit '1' is pressed, the oscillators will produce the following continuous-time signal
$$
    x(t) = \sin(2\pi\cdot 1209\cdot t) + \sin(2\pi\cdot697\cdot t)
$$

When dialing a multi-digit number, successive dial tones must be separated by a silent gap; although the official standard does not set standard timings, a DTMF receiver should be designed according to the following specifications:
 * valid dial tones can be as short as 40ms 
 * the silent gap between tones can also be as short as 40ms
 * actual tone frequencies can deviate up to $\pm 1.5\%$ from their nominal values 

## Digital to analog 

A digital-to-analog (D/A) converter, such as the soundcard in your PC, creates its analog output by interpolating the incoming digital samples at a rate of $F_s$ samples per second; this rate is the "clock" of D/A converter and, although it is an _interpolation_ rate, it is usually referred to as the _sampling rate_ or sampling _frequency_ of the system, using the same term that we use for an analog-to-digital converter.

When a soundcard with interpolation rate $F_s$ "plays" a discrete-time sequence of the form $x[n] = \cos(\omega_0 n)$ (that is, a discrete-time sinusoid with digital frequency $\omega_0 \in [-\pi, \pi]$), it outputs the continuous-time sinusoid $x(t) = \cos(2\pi f_0 t)$ where

$$
    f_0 = \frac{\omega_0}{2\pi}F_s. \tag{1}
$$

This means that the analog frequency of the output depends _both_ on the frequency of the discrete-time sinusoid _and_ on the interpolation rate of the soundcard, which is usually a design parameter. In general, we want to keep all sampling rates as low as possible since the power consumption of an D/A chip is approximately proportional to $F_s^2$. 

As an example, here you can listen to how the pitch changes when the _same_ discrete-time sinusoid is played by the soundcard at different interpolation rates (and note how the duration of the audio also changes, obviously): 

In [ ]:
w = 2 * np.pi * 0.05 
x = np.sin(w * np.arange(0, 8000))

for Fs in [8000, 16000, 4000]:
    print(f'Using an interpolation rate of {Fs} samples per second:')
    display(IPython.display.Audio(x, rate=Fs, normalize=False))

### Exercise: minimum sampling rate

What is the minimum value for the interpolation/sampling rate needed to implement a digital DTMF transmitter?

**SOLUTION:** 

Since the fastest possible digital frequency is $\omega = \pi$, as per eq. (1) the highest frequency that an D/A can generate is $F_s/2$. The largest frequency value in the DTMF table is 1477 Hz, and therefore we need 
$$
    F_s > 2954.
$$

## Analog to digital

The soundcard in your PC also works as an analog-to-digital (A/D) converter: it records an incoming audio signal by measuring (that is, by _sampling_) its amplitude $F_s$ times per second. 

If the input is a sinusoid of the form $x(t) = \sin(2\pi f_0 t)$ the resulting discrete-time signal will be 

$$
    x[n] = \sin(\omega_0 n) \qquad \text{with} \qquad \omega_0 = 2\pi\frac{f_0}{F_s}.
$$

As long as the a sampling frequency is larger than _twice_ the frequency of the input sinusoid, the sequence of samples is a perfect representation of the analog waveform in the sense that $x[n]$ can be interpolated back into $x(t)$ _exactly_ by a D/A converter also operating at $F_s$ samples per second.

### Exercise: aliasing

Consider D/A converter connected in cascade to a A/D converter; both converters operate at the same rate $F_s$. Assume that the input to the cascade is the signal $x(t) = \sin(2\pi f_0 t)$ with $f_0 = 1.6F_s$.

What is the frequency of the sinusoid at the output of the cascade?

**SOLUTION:** 

The A/D will produce a discrete time sinusoid $x[n] = \sin(\omega_0 n)$ whose digital frequency is
$$
    \omega_0 = 2\pi\frac{f_0}{F_s} = 3.2\pi.
$$
Since $\omega_0 > 2\pi$ we can use trigonometry to bring the frequency in the $[-\pi,\pi]$ interval:
$$
    x[n] = \sin(3.2\pi n) = \sin(1.2\pi n + 2\pi n) = \sin(1.2\pi n)
$$
When this signal enters the D/A, it produces an analog sinusoid with frequency
$$
    f'_0 = \frac{1.2\pi}{2\pi}F_s = 0.6F_s,
$$
that is, the original frequency has been aliased to a completely different value.

## Final design parameter

Although in theory we could use a lower value, in the rest of the notebook we will use $F_s = 8000$:
 * since the telephone channel is "natuarally" bandlimited to 4000 Hz, with a sampling frequency of 8 kHz no additional anti-aliasing filter is needed
 * in most soundcards, this is the lowest available sampling rate.

In [ ]:
Fs = 8000

# The encoder

In the next exercise, you task will be to implement a DTMF encoder as a Python function: the function takes a string of key values as input and returns their DTMF encoding as discrete-time audio signal that can be played at a rate of $F_s$ samples/second.

To get you started, here is a partial implementation where:
 * the DTMF frequency pairs are available as a dictionary, indexed by the key values
 * the durations (in seconds) of the tones and the silence gap are also specified.

## Exercise: implementation of a DTMF encoder

Complete the function below so that it returns the DTMF encoding of a series of key values, passed as a string. The encoding should be padded with 250 milliseconds of silence both at the beginning and at the end.

In [ ]:
def DTMF_encode(digits: str, Fs=8000) -> np.ndarray: 
    PADDING_SEC = 0.25
    TONE_SEC, SPACE_SEC = 0.2, 0.1
    DTMF_FREQS = {
        '1': (697, 1209), '2': (697, 1336), '3': (697, 1477),
        '4': (770, 1209), '5': (770, 1336), '6': (770, 1477),
        '7': (852, 1209), '8': (852, 1336), '9': (852, 1477),
        '*': (941, 1209), '0': (941, 1336), '#': (941, 1477),        
    }
    
    # index range for tone intervals
    #n = np.arange(...)
    
    #  output signal, start with initial silence
    #x = np.zeros(...))
    
    for k in digits:
        try:
            # select the DTMF frequencies
            ... 
            # append tones and space to output
            # x = np.r_[ x, ... ]
        except KeyError:
            print(f'invalid key: {k}')
            return None
    # append final silence and return
    return #...

In [ ]:
# SOLUTION:

def DTMF_encode(digits: str, Fs=8000) -> np.ndarray: 
    PADDING_SEC = 0.25
    TONE_SEC, SPACE_SEC = 0.2, 0.1
    DTMF_FREQS = {
        '1': (697, 1209), '2': (697, 1336), '3': (697, 1477),
        '4': (770, 1209), '5': (770, 1336), '6': (770, 1477),
        '7': (852, 1209), '8': (852, 1336), '9': (852, 1477),
        '*': (941, 1209), '0': (941, 1336), '#': (941, 1477),        
    }

    # index range for tone intervals
    n = np.arange(0, int(TONE_SEC * Fs))
    # output signal, start with initial silence
    x = np.zeros(int(PADDING_SEC * Fs))
    
    for k in digits:
        try:
            w_lo, w_hi = 2 * np.pi * np.array(DTMF_FREQS[k]) / Fs 
            x = np.r_[ x, np.sin(w_lo * n) + np.sin(w_hi * n), np.zeros(int(SPACE_SEC * Fs)) ]
        except KeyError:
            print(f'invalid key: {k}')
            return None
    return np.r_[ x, np.zeros(int((PADDING_SEC - SPACE_SEC) * Fs)) ]

Let's test it and evaluate it "by ear":

In [ ]:
x = DTMF_encode('123##45', Fs=Fs)
IPython.display.Audio(x, rate=Fs)

# The decoder

In telecommunication systems designing the encoder is usually the easy part; designing a robust decoder, on the other hand, is significantly more difficult since there are a lot of things that can degrade the quality of the received signal such as
 * additive background noise, and sometimes LOTS of noise
 * linear and nonlinear distortion
 * variable duration of tones and gaps
 * and much more.

In this notebook, however, we will only use clean signals and focus on the main time-frequency aspects of the decoding process. Our previous analysis using spectrograms suggests that, in order to decode a DTMF signal, we should first isolate each tone interval and then look for peaks in the local spectrum. 

## Signal segmentation

Since we know that DTMF tone intervals are separated by silent gaps, a simple strategy for signal segmentation could be the following:
 * estimate the *local energy* of the signal over time
 * mark the beginning and the end of each high-energy portions by looking at transitions in the local energy level 

### Exercise: compute the local energy

The local energy of the signal is the sum of the square magnitude of the samples within a short *analysis window* that slides over the entire signal. The length of the window should be large enough to avoid excessive fluctuations in the sequence of local energy values but it should also be sufficiently small so that silence gaps produce measurable drops in the local energy level. Since the DTMF standard allows for tones as short as 40ms, we should use a window about half that length.

Complete the following function so that it returns the sequence 
$
    e[n] = \sum_{k=-L/2}^{L/2} |x[n+k]|^2
$
where $L$ is the length in samples of the analysis window

In [ ]:
def local_energy(x: np.ndarray, win_len_ms: float=20, Fs: float=8000) -> np.ndarray:
    L =  ...
    e = np.zeros_like(x)
    for n in range(len(x)):
        e[n] = ...
    return e

In [ ]:
# SOLUTION

def local_energy(x: np.ndarray, win_len_ms: float=20, Fs: float=8000) -> np.ndarray:
    L =  int(win_len_ms * Fs / 1000 / 2)
    e = np.zeros_like(x)
    for n in range(len(x)):
        e[n] = np.sum(x[n-L:n+L] ** 2)
    return e

In [ ]:
loc_erg = local_energy(dtmf_signal) 
plt.plot(loc_erg);

### Thresholding

Now that we have a way to estimate the local of the signal, we can use this information to segment the audio file. Visually it seems obvious that we need to look for power transitions from low to high and vice-versa but, in practice, to determine if there was a transition we need to use a _threshold_, that is, a reference power value that we compare the current power level to. 

How can we choose the right value of the threshold? Things to think about:
 * the signal could be noisy, so the power in gaps between tones can be almost as large as the power in tones
 * in real life, we don't know the "volume" of the received signal: attenuation or amplification can arbitrarily change the amplitude of the sinusoids

In general, choosing a right value for the threshold is not trivial. Here, to keep things simple, let's just select the average local energy value.

In [ ]:
th = np.mean(loc_erg)
plt.plot(loc_erg);
plt.axhline(th, c='red');

The following function returns the start and stop chunk indexes associated to the DTMF tones.

In [ ]:
def tone_intervals(x: np.ndarray, win_len_ms: float=20, Fs: float=8000) -> list:
    erg = local_energy(x, win_len_ms, Fs)
    erg = erg > np.mean(erg)

    tones, a = [], 0
    for n in range(1, len(erg)):
        if erg[n]  and not erg[n-1]:
            a = n
        elif not erg[n] and erg[n-1]:
            tones.append((a, n))
            a = n
    return tones

In [ ]:
print(tone_intervals(dtmf_signal))

In [ ]:
plt.plot(dtmf_signal);
for tone in tone_intervals(dtmf_signal):
    plt.plot(np.arange(*tone), dtmf_signal[slice(*tone)]);

## Frequency identification

Once we have determined where the tone intervals are, we need to verify that they contain two distinct spectral lines (corresponding to the two tones associated to each key) and then recover the pressed key from the frequency values of the tones. Since we have already segmented the signal to compute the local energy, one robust way to do so is by computing the DFT of each chunk and accumulate the results in a frequency bin histogram over the duration of a tone interval; the two peaks will tell us the two tone frequencies.

The "low" DTMF frequencies are in the 697 Hz to 941 Hz range, while the high frequencies are in the 1209 Hz to 1477 Hz range. We will look for the lagest DFT value in each of those intervals. 

In [ ]:
def dtmf_key(dft: np.ndarray, Fs=8000):
    LO_FREQS = np.array([697.0, 770.0, 852.0, 941.0]) # DTMF rows
    HI_FREQS = np.array([1209.0, 1336.0, 1477.0])     # DTMF columns
    SPLIT_FREQ = 1075  # midpoint bewtween largest low and smallest high freq
    KEYS = [['1', '2', '3'], 
            ['4', '5', '6'], 
            ['7', '8', '9'], 
            ['*', '0', '#']] 

    N = len(dft)
    split = int(SPLIT_FREQ * N / Fs)
    # low frequency peak
    lo = np.argmax(dft[:split])
    r = np.argmin(np.abs(LO_FREQS - lo * Fs / N))
    # high frequency peak (only look over the first half of the DFT vector)
    hi = np.argmax(dft[split:N//2]) + split
    c = np.argmin(np.abs(HI_FREQS - hi * Fs / N))
    return KEYS[r][c]    

## Exercise: the final design

Complete the following function that implements a DTMF decoder:

In [ ]:
def DTMF_decode(x: np.ndarray, Fs=8000) -> str:
    WIN_LEN_MS = 20
    ...


In [ ]:
# SOLUTION

def DTMF_decode(x: np.ndarray, Fs=8000) -> str:
    WIN_LEN_MS = 20

    ret = ''
    for start, stop in tone_intervals(x, WIN_LEN_MS, Fs):
        ret += dtmf_key(np.abs(np.fft.fft(x[start:stop])))
    return ret

In [ ]:
for k in ['159', '915915', 'all']:
    Fs, x = read_test_file(f'keys_{k}.wav')
    print(f'decoding {k}: ', DTMF_decode(x))